In [1]:
from os import path

import torch
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import LazyConfig, instantiate
from diffusers import AutoPipelineForText2Image, DiffusionPipeline
from diffusers_interpret import StableDiffusionPipelineDetExplainer

/Users/zacbi/miniconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/zacbi/miniconda3/envs/llava/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/zacbi/miniconda3/envs/llava/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9A4710B9-0DA3-36BB-9129-645F282E64B2> /Users/zacbi/miniconda3/envs/llava/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <E459C462-F863-3A5A-AC9F-FD77B14BE845> /Users/zacbi/miniconda3/envs/llava/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong

In [ ]:
model_path = '/mnt/workspace/model'
eva_id = 'zacbi2023/eva02'
sd_id = 'AI-ModelScope/stable-diffusion-v1-5'

In [ ]:
# prepare eva
eva_base_path = path.join(model_path, eva_id)
eva_coco_config_rpath = 'projects/ViTDet/configs/eva2_o365_to_coco/eva2_o365_to_coco_cascade_mask_rcnn_vitdet_l_8attn_1536_lrd0p8.py'
eva_config_path = path.join(eva_base_path, eva_coco_config_rpath)

# replace with your eva02 weights path
eva_coco_weights_rpth = 'checkpoints/eva02_L_coco_seg_sys_o365.pth'
eva_weights_path = path.join(eva_base_path, eva_coco_weights_rpth)

custum_cfg = ['MODEL.RETINANET.SCORE_THRESH_TEST', 0.5,
                'MODEL.ROI_HEADS.SCORE_THRESH_TEST', 0.5,
                'MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH', 0.5,
                'DATASETS.TEST', [],
                'MODEL.WEIGHTS', eva_weights_path]
eva_cfg = LazyConfig.load(eva_config_path)
LazyConfig.apply_overrides(
    eva_cfg, [f"{key}={value}" for key, value in zip(custum_cfg[::2], custum_cfg[1::2])])

device = 'cuda'
eva = instantiate(eva_cfg.model).to(device)
DetectionCheckpointer(eva).load(eva_weights_path)
eva.eval()

In [ ]:
torch_dtype='float16'
sd_pipe = AutoPipelineForText2Image.from_pretrained(
    path.join(model_path, sd_id), torch_dtype=torch_dtype).to(device)
explainer = StableDiffusionPipelineDetExplainer(pipe=sd_pipe, det_model=eva)


In [ ]:
raw_emb = torch.load('')
gen_prefix_embs = torch.load('')

gen_text_hidden_fcs = torch.load('')
gen_emb = gen_text_hidden_fcs[0](raw_emb, gen_prefix_embs)
gen_emb = gen_text_hidden_fcs[0](raw_emb, gen_prefix_embs)


explainer(
    prompt_embeds=gen_emb,
    num_inference_steps=50,
    target_cls_id=15,
    raw_emb=raw_emb
)